# This is for the besties that want quick English bases hi there

i actually don't know how to setup MFA in here but we're just gonna have to figure that out and by we i mean Me DogeyVOX.

also Obviously This Is Gonna Be For English Only. so yeah.

# Setup Stuff

this is gonna install both Whisper and MFA so have fun.

In [ ]:
#@title Whisper install

!pip install -U openai-whisper

In [ ]:
#@title MFA install (Part 1)
%%writefile install_mfa.sh
#!/bin/bash

## a script to install Montreal Forced Aligner (MFA)

root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

#install MFA
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge montreal-forced-aligner -y

echo -e "\n======== DONE =========="
echo -e "\nTo activate MFA, run: source $root_dir/miniconda3/bin/activate aligner"
echo -e "\nTo delete MFA, run: rm -rf $root_dir"
echo -e "\nSee: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA"

In [ ]:
#@title MFA install (Part 2)
MFA_DIR = '/tmp/mfa'

!bash ./install_mfa.sh {MFA_DIR}
!source {MFA_DIR}/miniconda3/bin/activate aligner; mfa model download acoustic english_us_arpa; mfa model download dictionary english_us_arpa; mfa model download g2p english_us_arpa

In [ ]:
#@title Other install

!pip install tgt

In [ ]:
#@title Prepare functions

import tgt
from pathlib import Path
from abc import ABC, abstractmethod
import librosa

class PhonemeConverter(ABC):
    @abstractmethod
    def invoke(cur, prv=None, nxt=None):
        raise NotImplementedError('lol ur phoneme converter isn\'t working')

class ArpabetPlusSchwaPlusTapR(PhonemeConverter):
    def invoke(cur, prv=None, nxt=None):
        phone = cur.text.lower() # current phoneme
        if not phone: # it's silence if it doesn't exist
            return 'pau'

        if phone in 'dt': # turn d and t to dx if time is less than 0.04 seconds and surrounding phonemes are vowels
            if cur.end_time - cur.start_time < 0.04:
                if prv and nxt:
                    if prv.text and nxt.text:
                        if prv.text[-1].isnumeric() and nxt.text[-1].isnumeric():
                            return 'dx'

        # turn vowels into regular arpa and ah0 to schwa
        if phone[-1].isnumeric():
            phone = 'ax' if phone == 'ah0' else phone[:-1]
        return phone

class ArpabetPlusSchwa(PhonemeConverter):
    def invoke(cur, prv=None, nxt=None):
        phone = cur.text.lower() # current phoneme
        if not phone: # it's silence if it doesn't exist
            return 'pau'

        # turn vowels into regular arpa and ah0 to schwa
        if phone[-1].isnumeric():
            phone = 'ax' if phone == 'ah0' else phone[:-1]
        return phone

class RegularArpabet(PhonemeConverter):
    def invoke(cur, prv=None, nxt=None):
        phone = cur.text.lower() # current phoneme
        if not phone: # it's silence if it doesn't exist
            return 'pau'

        # turn vowels into regular arpa
        if phone[-1].isnumeric():
            phone = phone[:-1]
        return phone

class safelist(list):
    def get(self, index, default=None):
        try:
            return self.__getitem__(index)
        except IndexError:
            return default

def whisper_segment_to_interval(segment):
    # turn whisper segment into textgrid interval
    return tgt.core.Interval(segment['start'], segment['end'], segment['text'])

def whisper_words_to_intervals(segment):
    # turn whisper segment word timings into textgrid intervals
    words = []
    for word in segment['words']:
        words.append(tgt.core.Interval(word['start'], word['end'], word['word']))
    return words

def whisper_transcription_to_textgrid(wav_path, transcription, word_level=False):
    # turn whisper transcription to textgrid
    # get textgrid path
    textgrid_path = wav_path.with_suffix('.TextGrid')
    wav_length = librosa.get_duration(path=wav_path)

    # prepare textgrid and tier
    textgrid = tgt.core.TextGrid(str(textgrid_path))
    if not word_level:
        # make sentence tier
        sentence_tier = tgt.core.IntervalTier(end_time=wav_length, name='sentences')

        # convert segments to intervals and add
        sentence_tier.add_intervals(map(whisper_segment_to_interval, transcription['segments']))

        # add tier
        textgrid.add_tier(sentence_tier)
    else:
        # make word tier
        word_tier = tgt.core.IntervalTier(end_time=wav_length, name='words')

        # convert words from segments to intervals and add
        for segment in transcription['segments']:
            word_tier.add_intervals(whisper_words_to_intervals(segment))

        # add tier
        textgrid.add_tier(word_tier)

    # write file
    tgt.io.write_to_file(textgrid, str(textgrid_path), 'long')

def aligned_textgrid_to_lab(textgrid_path, phoneset='ArpabetPlusSchwaPlusTapR'):
    # turn MFA aligned textgrid to lab
    # get lab path
    lab_path = textgrid_path.with_suffix('.lab')

    # read textgrid
    textgrid = tgt.io.read_textgrid(str(textgrid_path), include_empty_intervals=True)
    phoneme_tier = textgrid.get_tier_by_name('phones')
    phonemes = safelist(phoneme_tier.annotations)
    converter = globals()[phoneset]

    # loop through all phonemes and write
    with open(lab_path, 'w', encoding='utf8') as f:
        for i in range(len(phonemes)):
            cur = phonemes[i]
            s = int(10000000 * cur.start_time)
            e = int(10000000 * cur.end_time)
            p = converter.invoke(cur, phonemes.get(i-1), phonemes.get(i+1))
            f.write(f'{s} {e} {p}\n')


In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.flush_and_unmount()
!rm -rf /content/drive
drive.mount('/content/drive')
print('Done!')

# Dataset Prep

i don't wanna do anything fancy so.


In [ ]:
#@title Extract DB

#@markdown Only accepting `.7z` or whatever `p7zip` reads sowwy...

#@markdown Folder format? nah just put wavs in the thing and it should be fine. it Will move everything in one folder tho so make sure there's no filename clashes.

#@markdown if you're doing vocal modes and u find it better to sort by folder make sure u can sort it back again easily. like idk do like `modename_###.wav` or smn.

#@markdown filename doesn't matter unless if you have transcriptions/timed transcriptions already the format is the same just make sure the wavs and the txt or textgrids have the same filename
import shutil

db_zip_loc = '/content/drive/MyDrive/db.7z' #@param {type: "string"}

db_zip_path = Path(db_zip_loc)
db_path = Path('db')
wav_path = db_path / 'wav'
lab_path = db_path / 'lab'
raw_path = Path('raw')

if wav_path.exists():
    shutil.rmtree(wav_path)

wav_path.mkdir(parents=True, exist_ok=True)
lab_path.mkdir(parents=True, exist_ok=True)

!7za x "$db_zip_loc" -o"raw"
for wav in raw_path.glob('**/*.wav'):
    shutil.move(wav, wav_path)

for txt in raw_path.glob('**/*.txt'):
    shutil.move(txt, wav_path)

for tg in raw_path.glob('**/*.TextGrid'):
    shutil.move(tg, wav_path)

raw_path.rmdir()

# Transcription and Forced Alignment

this is the fun part

In [ ]:
#@title Whisper Transcription

#@markdown this is for those who didn't add their own transcriptions/timed transcriptions

#@markdown ---

#@markdown ## Parameters
#@markdown like the first part says it's english only. `medium.en` gives pretty
#@markdown precise labels for word-level, but don't expect it to be that good.
#@markdown `base.en` is perfectly fine for sentence-level.
#@markdown i haven't tested the multilingual models.
model = 'medium.en' #@param ["tiny.en", "base.en", "small.en", "medium.en", "tiny", "base", "small", "medium", "large"]

#@markdown this is mostly experimental because word-level timing is a bit weird.
word_level_transcription = False #@param {type: "boolean"}

#@markdown just some funny params that i can expose but i don't exactly recommend changing ig
compression_ratio_threshold = 2.4 #@param {type: "number"}
log_prob_threshold = -1.0 #@param {type: "number"}
no_speech_threshold = 0.6 #@param {type: "number"}
condition_on_previous_text = True #@param {type: "boolean"}
verbose = True #@param {type: "boolean"}

import whisper

whisper_model = whisper.load_model(model)

for wav in wav_path.glob('*.wav'):
    print(f'Transcribing {wav}')
    transcription = whisper_model.transcribe(str(wav),
                                             compression_ratio_threshold=compression_ratio_threshold,
                                             logprob_threshold = log_prob_threshold,
                                             no_speech_threshold = no_speech_threshold,
                                             condition_on_previous_text=condition_on_previous_text,
                                             word_timestamps=word_level_transcription,
                                             verbose=verbose,
                                             language='en')

    whisper_transcription_to_textgrid(wav, transcription, word_level=word_level_transcription)

print('\nWhisper Transcription: Done.')

In [ ]:
#@title MFA Validate

#@markdown this is to check if mfa is being silly and not wanting to read some of ur wavs. if it misses some files i really don't know why it does so i can't help with that sorry.
!source {MFA_DIR}/miniconda3/bin/activate aligner; mfa validate --single_speaker --ignore_acoustics db/wav english_us_arpa

In [ ]:
#@title MFA Forced Alignment

#@markdown this is what turns the whisper transcriptions into silly phoneme-level labels. mind you this is trained on speech so it does kinda not slay sometimes.

#@markdown ---

#@markdown ## Parameters
#@markdown these are the only params i care about lol beam is how big the search is and retry beam is a bigger search when mfa thinks it flopped

beam_size = 1000 #@param {type: "integer"}
retry_beam_size = 4000 #@param {type: "integer"}
!source {MFA_DIR}/miniconda3/bin/activate aligner; \
mfa g2p db/wav english_us_arpa oov.txt --dictionary_path english_us_arpa; \
mfa model add_words english_us_arpa oov.txt; \
mfa align --fine_tune --single_speaker --textgrid_cleanup db/wav english_us_arpa english_us_arpa db/lab --beam {beam_size} --retry_beam {retry_beam_size}

In [ ]:
#@title TextGrid conversion

#@markdown this is to convert the format the MFA makes cuz duh we're in `.lab` gang

phoneset = 'regular arpabet' #@param ["regular arpabet", "arpabet + schwa", "arpabet + schwa + tap r"]
phoneset = ''.join(map(lambda x : x[0].upper() + x[1:], phoneset.replace('+', 'plus').split()))

for tg in lab_path.glob('*.TextGrid'):
    aligned_textgrid_to_lab(tg, phoneset=phoneset)

In [ ]:
#@title Save to Drive

#@markdown cuz drive downloads are faster.
keep_textgrids = False #@param {type: "boolean"}

if not keep_textgrids:
    for tg in db_path.glob('**/*.TextGrid'):
        tg.unlink()

labeled_db_path = db_zip_path.with_stem(db_zip_path.stem + '_labeled')
labeled_db_loc = str(labeled_db_path)
!7za a "$labeled_db_loc" "db"